In [26]:
import pandas as pd
import re
from lxml import etree
import shutil
import os
import math
import glob
import copy

In [2]:
def get_mapping_for_xml_invoice(file_path_of_sample_xml):

    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:Invoice-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2"
    #     'xmlns' : "urn:oasis:names:specification:ubl:schema:xsd:Invoice-2"
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
    # format_key_map = []
    invoice_dict={}
    for element in target_root.iter():
        ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

        string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

        path = ' / '.join(string_ancestors)
        for prefix, uri in namespace_map.items():
            path = path.replace('{' + uri + '}', prefix + ':')
        if 'sb:StandardBusinessDocument / ubl:Invoice / ' in path:
            path = path.replace('sb:StandardBusinessDocument / ', '')
            if element.attrib:
                xpath_base = path
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e

                    invoice_dict[xpath] = tree.getpath(element)
            else:
                invoice_dict[path] = tree.getpath(element)

        else:
            if element.attrib:
                xpath_base = get_element_xpath(element)
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
    #                 print(f'-->{xpath}')
                    invoice_dict[xpath] = tree.getpath(element)
            else:
                xpath = get_element_xpath(element)#defined on top
                invoice_dict[xpath] = tree.getpath(element)
    r_h = invoice_dict
    return r_h
def get_mapping_for_xml_credit(file_path_of_sample_xml):
    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:CreditNote-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2" 
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
        # format_key_map = []
    credit_dict={}
    for element in target_root.iter():
        if isinstance(element, etree._Comment):
            continue

        else:
            ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

            string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

            path = ' / '.join(string_ancestors)
            for prefix, uri in namespace_map.items():
                path = path.replace('{' + uri + '}', prefix + ':')

            if 'sb:StandardBusinessDocument / ubl:CreditNote / ' in path:

                path = path.replace('sb:StandardBusinessDocument / ', '')

                if element.attrib:
                    xpath_base = path
                    credit_dict[xpath_base] = tree.getpath(element)
                    for e,val in element.attrib.items():
                        xpath = xpath_base + ' / ' + '@' + e

                        credit_dict[xpath] = tree.getpath(element)
                else:
                    credit_dict[path] = tree.getpath(element)

            else:
                if element.attrib:
                    xpath_base = get_element_xpath(element)
                    credit_dict[xpath_base] = tree.getpath(element)
                    for e,val in element.attrib.items():
                        xpath = xpath_base + ' / ' + '@' + e
        #                 print(f'-->{xpath}')
                        credit_dict[xpath] = tree.getpath(element)
                else:
                    xpath = get_element_xpath(element)#defined on top
                    credit_dict[xpath] = tree.getpath(element)
    c_h = credit_dict
    return c_h
def credit_or_invoice(file_path):
#     print('********')
#     print(file_path)
    tree = etree.parse(file_path)
    target_root = tree.getroot()
    for index,e in enumerate(target_root.iter()):
        if index ==11:
            if 'Credit' in e.text:
                return 'credit'
            elif 'Invoice' in e.text:
                return 'invoice'
            else:
                return 'undefined'
def proper_mapping_final(file_path):

    if credit_or_invoice(file_path) == 'credit':
        r_h = get_mapping_for_xml_credit(file_path)
        return r_h
    elif credit_or_invoice(file_path) == 'invoice':
        r_h = get_mapping_for_xml_invoice(file_path)
        return r_h
    else:
        return None
def get_element_xpath(element):
    path = [element.tag.split('}')[-1]]
    for parent in element.iterancestors():
        parent_tag = parent.tag.split('}')[-1]  
        path.insert(0, parent_tag)
    return ' / '.join(path)
# def handle_attrib(atrib):#maybe store values just in case later ,i njust need the path in this case
#     for key in atrib:
#         key_val = key
#     key_val = '@' + key_val
#     return key_val
def read_xml_file(file_path_xml):
    result_xml_dict={}
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
#     pattern = r'\{.*?\}'
    for element in target_root.iter():
        if element.attrib:
            xpath_base = get_element_xpath(element)
            for e,val in element.attrib.items():
                xpath = xpath_base + ' / ' + '@' + e
#                 print(f'-->{xpath}')
                result_xml_dict[xpath] = tree.getpath(element)
        else:
            xpath = get_element_xpath(element)
            result_xml_dict[xpath] = tree.getpath(element)
    return result_xml_dict
        
def get_element_full_path(tree,element):
    return tree.getpath(element)

def read_xml_file_2(file_path_xml):
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
    for element in target_root.iter():
        xpath = tree.getpath(element)
        print(f"XPath: {xpath}")
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b55-1"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        print('creating file...')
        variable_to_store = "11e1d6e4-74ba-49cc-9b55-1"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')
def adjust_identifier(root_copy,filepath_base_identifier):
    base_identifier = load_base_value_iden(filepath_base_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 10:#rsender identifier:
            e.text = base_identifier
            increment_value_iden(base_identifier,filepath_base_identifier)
def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp

def handle_base_path(s):
    start = r_h[s]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
def fix_space_typo(temp):
    temp_2 = temp.split("/")
    for index,i in enumerate(temp_2):
        temp_2[index] = i.strip()
    temp_3 = ' / '.join(temp_2)
    return temp_3

In [65]:
root_path = '/Users/cheongray/iras_api_clean_1'

In [66]:
file_path = r"/Users/cheongray/iras_api_clean_1/AP_SP_files/AP_SP_FINAL_5.xlsx"

os.path.join(root_path,'AP_SP_files','AP_SP_FINAL_5.xlsx')

'/Users/cheongray/iras_api_clean_1/AP_SP_files/AP_SP_FINAL_5.xlsx'

In [8]:
output_folder = os.path.join(root_path,'3_code_validation')



file_path = r"/Users/cheongray/iras_api_clean_1/AP_SP_files/AP_SP_FINAL_5.xlsx"#download to here from emails
path_to_sample_xml_files_folder = r"/Users/cheongray/iras_api_clean_1/sample_xml_files"
filepath_base_identifier = '/Users/cheongray/iras_api_make_xml/identifier_store/stored_base_identifier_conditional_validation.txt'


os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

output_file = os.path.join(output_folder, 'df_full.csv')
output_file_original = os.path.join(output_folder, 'df_full_back_up.csv')
if os.path.exists(output_file):
    os.remove(output_file)
if os.path.exists(output_file_original):
    os.remove(output_file_original)



sheet_name = '3_Code Validation'
df_s = pd.read_excel(file_path, sheet_name=sheet_name , keep_default_na=False, na_values=['NA'])
df_s_temp = df_s.copy()
df_s_temp = df_s_temp.fillna('NA')
df_s_temp['Test Case ID'] = df_s_temp.index.astype(float)#REMINDER,now test_case_id is 1_1
df_s_temp['Test Case ID'] = df_s_temp['Test Case ID'] + 1
df_s_temp.to_csv(output_file, index=False)
df_s_temp.to_csv(output_file_original, index=False)

In [9]:
#store the excel file 
df_s_temp_subset = df_s_temp.copy()

unique_descriptions = set()

target_elements_syn = []

for index,row in df_s_temp_subset.iterrows():
    filled_columns = row.dropna().index
    test_case_id = f' {index}'
    descript = row['Test Case Description']
#     print(f'Test case {index+1}')
#     print(f'Test case description : {descript}')
    unique_descriptions.add(descript)#just to build function_mapping
    temp={}
    temp['Rule ID'] = row['Rule ID']
    temp['Standard'] = row['Standard']
    temp['Document Type'] = row['Document Type']
    temp['BIS/PINT'] = row['BIS/PINT']
    temp['Test Case ID'] = int(row['Test Case ID'])
    id = int(row['Test Case ID'])
    temp['descript'] = descript
    temp['Expected Result'] = row['Expected Result Response (for 400 response)']
    temp['Status'] = row['Expected Result']
    temp['Fields'] = []
    temp['Sample'] = row['Sample']
    for i, col in enumerate(filled_columns, start=1):
        field_name = f'Field {i}'
        if field_name in df_s_temp_subset.columns:
            # Process the data as needed
#             print(f'Column: {field_name}, Value: {row[field_name]}')
   
            
            temp_2 = [] #each field will be one array,can do dict but mor steps when want access
            if not pd.isna(row[field_name]):
                field_value = field_name + ' Value'
                field_name_insert = fix_space_typo(row[field_name])
                temp_2.append(field_name_insert)
                temp_2.append(row[field_value])
                if field_name_insert == '':
                    continue
                else:
                    temp['Fields'].append(temp_2)
        
    target_elements_syn.append(temp)

            


In [10]:

#gets the needed files for 2_Conditional_Input_Validation
file_paths_xml_files = {}

for i,test_case in enumerate(target_elements_syn):
    
    sample_to_use = test_case['Sample']
    
    num = sample_to_use.split(' ')[-1]
    file_name = 'sample' + '_' + num + '.xml'
    
    path_to_sample_xml_files = path_to_sample_xml_files_folder + '/' + file_name
    
    file_paths_xml_files[sample_to_use] = path_to_sample_xml_files


In [11]:
# v='/*/*[2]/cac:TaxTotal/cac:TaxSubtotal[2]/cac:TaxCategory[2]/cbc:ID'
# target_prefix = v.split('/')[-1]
# len_t = len(target_prefix)
# print(target_prefix)

In [12]:
def adjust_cac(input_string):
    a='tax[2]'
    b = str(int(a[-2])-1)
    prefix = a[:-2]
    return prefix + b + a[-1]



In [52]:
def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            print("----")
            print("--customisation--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")

            
        
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:
            print("----")
            print("--profile--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")


def insert_for_attribute(target_element,insert_value,each_path):
    print('   ')
    print('------change attrib------')
    if len(target_element)>0:

        print(f'----<<--actual element value==>{target_element[0].attrib} type = {type(target_element[0].attrib)}')
        for key, value in target_element[0].attrib.items():
            print(f'val = {value}')
            target_element[0].attrib[key] = insert_value
        print(f'----<<--new attribute value==>{target_element[0].attrib}')

    else:
         print(f'--cannot get--<<--{each_path} | {target_element} is empty ')
    return target_element
def adjust_cac(input_string):
    a='tax[2]'
    b = str(int(a[-2])-1)
    prefix = a[:-2]
    return prefix + b + a[-1]

def insert_for_element(target_element,insert_value,each_path,actual_xpath):
    
    print('   ')
    print('------change element------')
    if len(target_element)>0:
        print(f'----<<--actual element text==>{target_element[0].text}')
        target_element[0].text = insert_value
        
        print(f'----<<--new element text==>{target_element[0].text}')
#         tree = etree.ElementTree(original_tree_copy)
# #         indexed_filename =sheet_name + 'Test case ID_' + str(test_case_id) + '_' + rule_id + '.xml'
#         indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
#         filepath = os.path.join(folder_name, indexed_filename)
    else:
        print(f'---cannot get-<<--{each_path} | {target_element} is empty ')
    return target_element
#still there
#takes in desc and fieldone path.resolves into xpath
#remove element if still there at the end for filed does not exist
#change from 'nan' to '' if field exist with missingn value,adjust value_store first element as thats waht
#field description is referring to

def delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path):
    attrib_yes = False
  
    if '@' in each_path:
        attrib_yes = True
        
    if 'Field does not exist' in desc:
        print(f'%%%%%%%%xpath find{actual_xpath_absolute} %%%%%%%%')
        print('                 ')
        print('Targeting element......')
        element_temp = target_root_copy.xpath(actual_xpath_absolute, namespaces=namespace_map)
        if len(element_temp)>0:
            element = element_temp[0]
            if attrib_yes:
                print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
                del element.attrib[key_attrib]
                print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
            else:
                print('%%%%%%%% Remove element %%%%%%%%')
                parent = element.getparent()

                if element.getparent() is not None:
                    element.getparent().remove(element)

                print('------------')
        else:
            print('xpath ')
        return target_root_copy
    else:
        print('%%%%%%%no attribute to remove -function : delete_element_attrib exiting%%%%%%%%%')
        print('                  ')
        return target_root_copy


def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp
def handle_base_path(s,r_h):
    start = r_h[s]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
            
def evaluate_if_got_multiple(current_test_case):
    occur_again = False
    for index,fields in enumerate(current_test_case):
        xpath_find = fields[0].strip()
        if index == 0:
            field_one = xpath_find
        if (xpath_find == field_one) and index != 0:
            occur_again = True
            return True
    return occur_again
def prepare_data_for_multiple(current_test_case):
    holding = []
    values_store=[]
    for index,fields in enumerate(current_test_case):
        
        path_to_store = fields[0]
        value = fields[1]
        values_store.append(value)
        if index == 0:
            temp = []
            
            field_one = path_to_store
            temp.append(path_to_store)
            
        elif (path_to_store == field_one) and (index != 0):
            holding.append(temp)
            temp = []

        
            temp.append(path_to_store)
        else:
            temp.append(path_to_store)
    holding.append(temp)
    return holding , values_store
def add_index_before_slash(input_string,index):    
    
    slash_index = input_string.find('/')
    
    # Check if '/' is found
    if slash_index == -1:
        # Return the original string if no '/' is found
        return input_string
    
    # Insert '[1]' before the first '/'
    if re.search(r'\[\d+\]', input_string):
        # Replace the existing '[]' with the new index
        modified_string = re.sub(r'\[\d+\]', f'[{index}]', input_string)
    else:
        # Insert '[index]' before the first '/'
        modified_string = input_string[:slash_index] + f'[{index}]' + input_string[slash_index:]
    
    return modified_string

def cac_count(s,substr):
    return s.count(substr)

def fix_get_second_cac(s,index,helper):
    
    if helper != 'Special' or cac_count(s,'cac') <= 1:
        print('no changees.......')
        return s
    else:
        t_split = s.split('cac')
        to_fix=t_split[-2]
        t_split[-1] = add_index_before_slash(t_split[-1],1)
        t_split[-2] = add_index_before_slash(to_fix,index)

        return 'cac'.join(t_split)

def handle_multiple(current_test_case,test_case_id,r_h,target_root,desc,helper):
    base_path_field_1_a = []
    whole_count_pos = 0
    hold_1 , value_store = prepare_data_for_multiple(current_test_case)
    field_1 = hold_1[0][0]
    base_path_field_1 = handle_base_path(field_1,r_h)
    base_path_field_1_a.append(handle_base_path(field_1,r_h))
    value_store_key = 0
    temp_base_path = base_path_field_1
    target_root_copy = copy.deepcopy(target_root)
    

    
    for index,each_case in enumerate(hold_1):
        
        

        path_base = temp_base_path + f'[{index+1}]' + '/'

        for each_path in each_case:
            
            insert_value = str(value_store[value_store_key])
            
            print(f'insert value = {value_store[value_store_key]}')
            if 'Field exists with missing value' in desc and index == 0:
                
                insert_value = ''
            value_store_key +=1
      

            base_path_each_path = handle_base_path(each_path,r_h)
            if base_path_each_path == base_path_field_1:
                if each_path in r_h:
                    actual_xpath = r_h[each_path]
                    temp = extract_out_second_half(actual_xpath)
                    
                    actual_xpath_absolute = path_base + temp
#                     actual_xpath_absolute = fix_get_second_cac(actual_xpath_absolute,index+1,helper)
#                     print(f'----<<--actual amended base path ==>{actual_xpath_absolute}')
                    
                    target_element = target_root_copy.xpath(actual_xpath_absolute,namespaces = namespace_map)

                    if '@' in each_path:
#                         print(f'>>>>>>target_element.attrib = {target_element[0].attrib}')
#                         print(f'>>>>>>target_element.text = {target_element[0].text}')
                        target_element = insert_for_attribute(target_element,insert_value,each_path)
                    else:
                        target_element = insert_for_element(target_element,insert_value,each_path,actual_xpath_absolute)
                    if whole_count_pos == 0:
#                         print(']]]]]]]]]]]]]]]]]base_path_each_path == base_path_field_1]]]]]]]]]]]]]]]]]]')
#                         print(f'each path = {each_path}')
#                         print(f'{actual_xpath_absolute}')
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path)
                        print('<<<<last>>>')
      

            else:#base_path_each_path != base_path_field_1:
#                     print('   ')
#                     print(f'----<<--just change straight away {each_path} index == {whole_count_pos}->>')
#                     print('   ')
                    if each_path in r_h:
                        actual_xpath = r_h[each_path]
                    
                    target_element = target_root_copy.xpath(actual_xpath,namespaces = namespace_map)

                    if '@' in each_path:
                        target_element = insert_for_attribute(target_element,insert_value,each_path)

                    else:
                        target_element = insert_for_element(target_element,insert_value,each_path,actual_xpath)
                    if whole_count_pos == 0:
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath,r_h)
                        print('<<<<last>>>')

#
            whole_count_pos += 1

#     print(value_store_key)
#     print('             ')
#     print(value_store)
    return target_root_copy


In [53]:
folder_name = output_folder + '/xml_files_for'+'_'+sheet_name

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)
    
namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }
  
def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            e.text = value

  
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:

            e.text = value

c=0
error_prone_files = []
d=0
error_in_file = []
error_in_file_store = []
files_made = []
attrib=0
for i,test_case in enumerate(target_elements_syn):
    
    current_test_case = test_case['Fields']
    test_case_id = test_case['Test Case ID']
    desc = test_case['descript']
    sample_to_use = test_case['Sample']
    standard  = test_case['Standard']
    biz_pint = test_case['BIS/PINT']
    rule_id = test_case['Rule ID']
#     helper = test_case['Helper']
    f_p = file_paths_xml_files[sample_to_use]
    print(f'###################{test_case_id}##########################')
    print(' ')
    print(f'{test_case_id}  using {f_p}')
    r_h = proper_mapping_final(f_p)
#     print(f'==helper_value===={helper}===={type(helper)}=')
    if not r_h:
        print(f'cannot make r_h : {test_case_id}  sample :{sample_to_use}')
        continue
    tree = etree.parse(f_p)
    target_root = tree.getroot()
#     print(f'test case id :{test_case_id}')
#     print(desc)
    original_tree_copy = copy.deepcopy(target_root)
    if standard == 'PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1'
            adjust_customisation_id(original_tree_copy,value_cus)

            
    if standard == 'Non-PEPPOL':
        print(f'test case id {test_case_id}' )
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)
    adjust_identifier(original_tree_copy,filepath_base_identifier)
    lenght_of_fields = len(current_test_case)
    occur_again = evaluate_if_got_multiple(current_test_case)
    if occur_again:
        
        print('ocur again ')
        c+=1
        temp = []
        temp.append(test_case['Test Case ID'])
        temp.append(test_case['descript'])
        print(' ')
        error_prone_files.append(temp)

        target_root = handle_multiple(current_test_case,test_case_id,r_h,original_tree_copy,desc,'temp')
        
        
        tree = etree.ElementTree(target_root)
        indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
        filepath = os.path.join(folder_name, indexed_filename)
        print(f'writing {indexed_filename} to folder......')
        tree.write(filepath, encoding='utf-8', xml_declaration=True)
        
    else:
        print('not occur again')
        d+=1
        temp = {}
        temp[test_case_id] = []
        for index,fields in enumerate(current_test_case):
            attrib_yes = False
            xpath_find = fields[0].strip()

            field_value = fields[1]
            if pd.isna(field_value):
                print('    ')
                print('++is NAN !!!!!!!!++')
                print('     ')

            if '@' in xpath_find:
                    attrib_yes = True

            if xpath_find in r_h:
                xpath_target = r_h[xpath_find]
                print(f'====={xpath_find}=======')
                if original_tree_copy.xpath(xpath_target, namespaces=namespace_map):
                    element = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)
                    if not attrib_yes:
                        element[0].text = str(field_value)

                    else:
                        insert_for_attribute(element,str(field_value),xpath_find)
            else:#+++
                
                temp[test_case_id].append(xpath_find)
                print(f'--->test case:{test_case_id} | uses {sample_to_use} key not found for {xpath_find}')
        if len(temp[test_case_id])>=1:
            error_in_file.append(temp[test_case_id])
                
    tree = etree.ElementTree(original_tree_copy)
#         indexed_filename =sheet_name + 'Test case ID_' + str(test_case_id) + '_' + rule_id + '.xml'
    indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
    filepath = os.path.join(folder_name, indexed_filename)
    
    tree.write(filepath, encoding='utf-8', xml_declaration=True)
    files_made.append(test_case_id)
print('c=',c)



###################1##########################
 
1  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml
not occur again
=====ubl:Invoice / cbc:InvoiceTypeCode=======
###################2##########################
 
2  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml
not occur again
=====ubl:Invoice / cbc:InvoiceTypeCode=======
###################3##########################
 
3  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml
not occur again
=====ubl:Invoice / cbc:InvoiceTypeCode=======
###################4##########################
 
4  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml
not occur again
=====ubl:Invoice / cbc:InvoiceTypeCode=======
###################5##########################
 
5  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml
not occur again
=====ubl:Invoice / cbc:InvoiceTypeCode=======
###################6##########################
 
6  using /Users/cheongray/iras_

not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################62##########################
 
62  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################63##########################
 
63  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################64##########################
 
64  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################65##########################
 
65  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################66##########################
 
66  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:Docume

not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################122##########################
 
122  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################123##########################
 
123  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################124##########################
 
124  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################125##########################
 
125  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################126##########################
 
126  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / 

not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################183##########################
 
183  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################184##########################
 
184  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################185##########################
 
185  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################186##########################
 
186  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################187##########################
 
187  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / 

not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################248##########################
 
248  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################249##########################
 
249  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################250##########################
 
250  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################251##########################
 
251  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / cbc:DocumentCurrencyCode=======
###################252##########################
 
252  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml
not occur again
=====ubl:Invoice / 

...

test case id 3040
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:Invoice / cbc:ProfileID=======
###################3041##########################
 
3041  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml
test case id 3041
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:Invoice / cbc:ProfileID=======
###################3042##########################
 
3042  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml
test case id 3042
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:Invoice / cbc:ProfileID======

test case id 3076
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:Invoice / cbc:ProfileID=======
###################3077##########################
 
3077  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml
test case id 3077
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:Invoice / cbc:ProfileID=======
###################3078##########################
 
3078  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml
test case id 3078
not occur again
=====ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:Invoice / cbc:ProfileID======

ocur again 
 
insert value = SRLVG
   
------change element------
----<<--actual element text==>ZR
----<<--new element text==>SRLVG
%%%%%%%no attribute to remove -function : delete_element_attrib exiting%%%%%%%%%
                  
<<<<last>>>
insert value = SC
   
------change element------
---cannot get-<<--ubl:CreditNote / cac:CreditNoteLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID | [] is empty 
insert value = SRLVG
   
------change element------
----<<--actual element text==>ZR
----<<--new element text==>SRLVG
insert value = SC
   
------change element------
----<<--actual element text==>SRLVG
----<<--new element text==>SC
insert value = urn:fdc:peppol.eu:2017:poacc:billing:01:1.0 
   
------change element------
----<<--actual element text==>urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
----<<--new element text==>urn:fdc:peppol.eu:2017:poacc:billing:01:1.0 
writing IRASC5-066_Test_case_3123_3_Code Validation.xml to folder......
###################3124#########################

not occur again
=====ubl:CreditNote / cac:CreditNoteLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:CreditNote / cbc:ProfileID=======
###################3175##########################
 
3175  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml
test case id 3175
not occur again
=====ubl:CreditNote / cac:CreditNoteLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:CreditNote / cbc:ProfileID=======
###################3176##########################
 
3176  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml
test case id 3176
not occur again
=====ubl:CreditNote / cac:CreditNoteLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:CreditNote / cbc

test case id 3226
not occur again
=====ubl:CreditNote / cac:CreditNoteLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:CreditNote / cbc:ProfileID=======
###################3227##########################
 
3227  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml
test case id 3227
not occur again
=====ubl:CreditNote / cac:CreditNoteLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ubl:CreditNote / cbc:ProfileID=======
###################3228##########################
 
3228  using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml
test case id 3228
not occur again
=====ubl:CreditNote / cac:CreditNoteLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID=======
=====ubl:CreditNote / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID=======
=====ub

In [54]:
file_paths_xml_files

{'Sample 1': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml',
 'Sample 3': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml',
 'Sample 2': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml',
 'Sample 4': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_4.xml',
 'Sample 11': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml',
 'Sample 9': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_9.xml',
 'Sample 10': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_10.xml'}

In [55]:
# # a='/*/*[2]/cac:TaxTotal/cac:TaxSubtotal[2]/cac:TaxCategory[1]/cbc:ID'
    
# namespace_map = {
#     'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
#     'xs': 'http://www.w3.org/2001/XMLSchema',
#     'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
#     'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
#     }
# a='/*/*[2]/cac:InvoiceLine[2]/cac:Price/cac:AllowanceCharge/cbc:BaseAmount'
# tree = etree.parse('/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml')
# target_root = tree.getroot()
# element = target_root.xpath(a, namespaces=namespace_map)
# element

In [56]:
# if 1 in error_in_file_store:
#     print(error_in_file_store[1])print(f'problems in {len(error_in_file)} files')
print(f'problems in {len(error_in_file_store)} files')

print(f'created in total {len(files_made)} files')
correct_files = [key for key in files_made if key not in error_in_file_store]
number_of_files_skipped = len(error_prone_files)
print(f'error prone files skipped {number_of_files_skipped}')
print(f'created {len(correct_files)} files with no issues')

problems in 0 files
created in total 3232 files
error prone files skipped 48
created 3232 files with no issues


In [57]:
# error_prone_files
to_manual_fix = []
for i in error_prone_files:
    to_manual_fix.append(i[0])

In [58]:
import numpy as np

array = np.array(to_manual_fix )
folder_name_store = '/Users/cheongray/iras_api_make_xml/store_manual_fix_files'
if not os.path.exists(folder_name_store):
    os.makedirs(folder_name)
new_file_path = os.path.join(folder_name_store, '3_code_validation_manual_fix_files.txt')
# Save the array to a text file
np.savetxt(new_file_path, array, fmt='%d')
new_file_path

'/Users/cheongray/iras_api_make_xml/store_manual_fix_files/3_code_validation_manual_fix_files.txt'

In [59]:
##### 

In [60]:
folder_name

'/Users/cheongray/api_clean_1/3_code_validation/xml_files_for_3_Code Validation'

In [61]:
files_to_fix_path = new_file_path
folder_path = folder_name
with open(files_to_fix_path, 'r') as file:
    lines_array = file.readlines()

problems = [int(line.strip()) for line in lines_array]
def get_id(xml_file):
    temp = xml_file.split('/')[-1]
    temp_2 = temp.split('_Test_case_')[-1].split('_')[0]
    return temp_2
def process_xml_files_3(folder_path):
    c=0
    map_to_values ={}#so can overwrite the dirty files in the oirinal 3_condtionial
    check_how_many_removed = []
    xml_files = glob.glob(os.path.join(folder_path, '*.xml'))
    for file in xml_files:
        test_id = get_id(file)
        if int(test_id) in problems:
            c+=1
            map_to_values[test_id] = file
    return map_to_values
mtv = process_xml_files_3(folder_path)


In [62]:
work_st_1 = []
for testcase in target_elements_syn:
    if testcase['Test Case ID'] in problems:
        temp=[]
        temp.append(testcase['Test Case ID'])
        temp.append(testcase['Fields'])
        temp.append(testcase['Sample'])
        work_st_1.append(temp)
work_st_2 = []
for index_work_1,e in enumerate(work_st_1):
    if int(work_st_1[index_work_1][0]) in problems:
        work_st_2.append(work_st_1[index_work_1])

In [63]:
namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }
for test_case in work_st_2:
    
    sample_number = test_case[2]
    test_case_id = test_case[0]
    f_p = mtv[str(test_case_id)]
    print(f'===={sample_number}====={f_p}={test_case_id}==')
    a = proper_mapping_final(f_p)
    holding_set = []
    values = []
    for path in test_case[1]:
        xpaths = a[path[0]]
        value = path[1]
        values.append(value)
        print( ' xpaths')
        print(xpaths)
        print('========')
        print(path)
        if isinstance(xpaths, str):
            if xpaths not in holding_set:
                 holding_set.append(xpaths)
        else:
            for x in xpaths:
                if x not in holding_set:
                    holding_set.append(x)
        
    print('          ')
    print(holding_set)
    tree = etree.parse(f_p)
    target_root = tree.getroot()
    print('   ')
    original_tree_copy = copy.deepcopy(target_root)
    for index_values,path_to_insert in enumerate(holding_set):
        
        #assume all element for now
        element =  original_tree_copy.xpath(path_to_insert,namespaces = namespace_map)[0]
        print(f'original text {element.text}')
        element.text = str(values[index_values])
        print(f'new text {element.text}')
        print('   ')
    tree = etree.ElementTree(original_tree_copy)
    filepath = mtv[str(test_case_id)]
    tree.write(filepath, encoding='utf-8', xml_declaration=True)
    
    print('============')

====Sample 9=====/Users/cheongray/api_clean_1/3_code_validation/xml_files_for_3_Code Validation/IRASC5-045_Test_case_2858_3_Code Validation.xml=2858==
 xpaths
/*/*[2]/cac:TaxTotal/cac:TaxSubtotal[2]/cac:TaxCategory/cbc:ID
['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID', 'SRCA-S']
 xpaths
/*/*[2]/cac:TaxTotal/cac:TaxSubtotal[2]/cac:TaxCategory/cbc:ID
['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID', 'ZR']
 xpaths
/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID
['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID', 'SRCA-S']
 xpaths
/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID
['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID', 'ZR']
 xpaths
/*/*[2]/cbc:ProfileID
['ubl:Invoice / cbc:ProfileID', 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0 ']
          
['/*/*[2]/cac:TaxTotal/cac:TaxSubtotal[2]/cac:TaxCategory/cbc:ID', '/*/*[2]/cac:Invo

   
original text ZR
new text SRLVG
   
original text ZR
new text SC
   
original text urn:peppol:bis:billing
new text SRLVG
   
====Sample 9=====/Users/cheongray/api_clean_1/3_code_validation/xml_files_for_3_Code Validation/IRASC5-066_Test_case_3088_3_Code Validation.xml=3088==
 xpaths
/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID
['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID', 'T-X']
 xpaths
/*/*[2]/cac:InvoiceLine[2]/cac:Item/cac:ClassifiedTaxCategory/cbc:ID
['ubl:Invoice / cac:InvoiceLine / cac:Item / cac:ClassifiedTaxCategory / cbc:ID', 'TX-R']
 xpaths
/*/*[2]/cac:TaxTotal/cac:TaxSubtotal[2]/cac:TaxCategory/cbc:ID
['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID', 'T-X']
 xpaths
/*/*[2]/cac:TaxTotal/cac:TaxSubtotal[2]/cac:TaxCategory/cbc:ID
['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cac:TaxCategory / cbc:ID', 'TX-R']
 xpaths
/*/*[2]/cbc:ProfileID
['ubl:Invoice / cbc:ProfileID', 'urn:peppol:bis: